In [25]:
import pandas as pd
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup

In [26]:
mypath = "NewList7/current sales"
files = [mypath+'/'+f for f in listdir(mypath) if isfile(join(mypath, f))]
files.sort()
files = files[0:100]


In [27]:
for file in tqdm(files):
    if '.csv' not in file:
        continue
    try:
        data = pd.read_csv(file)
    except:
        print(file)

    # loop through data
    #for index, row in tqdm(data.iterrows(), total=data.shape[0]):
    for index, row in data.iterrows():
        link = row['link']
        
        # Detailed Scraping
        try:
            deeper_info = BeautifulSoup(requests.get(link).content, 'html.parser')
        except:
            print("couldnt scrap")
            print(link)
            continue

        # Get image link
        try:
            image = deeper_info.find('img', id='icImg').get('src')
        except:
            try:
                image = deeper_info.find_all('img', class_='img')[0].get('src')
            except:
                print("no image")
                print(link)
                image = 'NA'

        # Get sales type
        try:
            num_bids = deeper_info.find('a', id='vi-VR-bid-lnk').find('span', id='qty-test').text.strip()
            sell_type = 'Auction'
            num_stock = 1
        except:
            try:
                num_bids = deeper_info.find('span', id='qty-test').text.strip()
                sell_type = 'Auction'
                num_stock = 1
            except:
                sell_type = 'Instant'
                num_bids = 0
                try:
                    num_stock = deeper_info.find('span', id='qtySubTxt').find_all('span')[0].text.strip().replace(' available','')
                except:
                    num_stock = 1


        # save data
        data.loc[index,'image'] = image
        data.loc[index, 'sell_type'] = sell_type
        data.loc[index, 'num_bids'] = num_bids
        data.loc[index, 'num_stock'] = num_stock
    
    data.to_csv(file,index=False)

 53%|█████▎    | 53/100 [1:35:00<51:20, 65.55s/it]   

no image
https://www.ebay.com/itm/334011194695?epid=22036981593&hash=item4dc49e5d47:g:DfwAAOSw~TJgqm3e


100%|██████████| 100/100 [1:47:15<00:00, 64.36s/it]  
